In [36]:
"""Citadel‑Grade Portfolio Risk Dashboard
======================================
*One‑file script → prints a readable, line‑by‑line risk sheet.*

Key parameters
--------------
YEARS, CONF_LVL, PORTF_VALUE_USD, WEIGHTS, PROXY.

Guarantees
----------
• Any funded ticker missing data aborts.  
• Only pre‑IPO gaps filled (no smoothing halts).  
• Final dashboard prints one metric per line.
"""
from __future__ import annotations
import pandas as pd, numpy as np, yfinance as yf
import logging, contextlib, io

# ────────── 1) USER SETTINGS ──────────
YEARS           = 6
CONF_LVL        = 0.95
PORTF_VALUE_USD = 100_000
RISK_FREE_RATE  = .05

WEIGHTS = {
    "BRK-B": 0.15, "META": 0.10, "GOOGL": 0.10, "QQQM": 0.05,
    "RDDT": 0.15, "HOOD": 0.075, "NBIS": 0.075, "SNOW": 0.075,
    "INTU": 0.075, "SHOP": 0.075, "TQQQ": 0.075,
    "daisy1": 0.0, "daisy2": 0.0, "daisy3": 0.0, "daisy4": 0.0, "daisy5": 0.0,
}
BENCH_SPY, BENCH_QQQ = "SPY", "QQQ"

PROXY = {
    "RDDT": [("SNAP", .4), ("GOOGL", .3), ("PLTR", .15), ("HOOD", .15)],
    "HOOD": [("COIN", .35), ("CBOE", .25), ("SQ", .15), ("MSTR", .15), ("PLTR", .10)],
    "NBIS": [("SMCI", .35), ("SNOW", .25), ("PLTR", .2), ("NET", .1), ("PATH", .1)],
    "SNOW": [("MDB", .3), ("DDOG", .25), ("NET", .2), ("PLTR", .15), ("MSFT", .1)], 
    "SHOP": [("SQ", .35), ("WIX", .25), ("PLTR", .2), ("MELI", .2)]
}

# ────────── 2) HELPERS ──────────
logging.getLogger("yfinance").setLevel(logging.CRITICAL)
TRADING_DAYS_YR = 252

@contextlib.contextmanager
def silence():
    with contextlib.redirect_stdout(io.StringIO()):
        yield

def adj_close(df: pd.DataFrame, t: str) -> pd.Series:
    close = df.xs("Close", level=0, axis=1).squeeze("columns") if isinstance(df.columns, pd.MultiIndex) else df["Close"]
    close.name = t
    return close

# ────────── 3) DOWNLOAD PRICES ──────────
window = int(YEARS * TRADING_DAYS_YR)
start  = pd.Timestamp.today().normalize() - pd.tseries.offsets.BDay(window + 40)
end    = pd.Timestamp.today().normalize()

proxy_members = {m for v in PROXY.values() for m in ([v] if isinstance(v, str) else [x for x, _ in v])}
universe = set(WEIGHTS) | {BENCH_SPY, BENCH_QQQ} | proxy_members

prices: dict[str, pd.Series] = {}
for tic in universe:
    with silence():
        raw = yf.download(tic, start=start, end=end, auto_adjust=True, progress=False, threads=False)
    if raw.empty:
        if WEIGHTS.get(tic, 0) > 0:
            raise RuntimeError(f"Missing data for funded ticker {tic}")
        continue
    prices[tic] = adj_close(raw, tic)

if BENCH_QQQ not in prices:
    raise RuntimeError("QQQ data missing")

df = pd.concat(prices.values(), axis=1)
qqq_series = df[BENCH_QQQ].ffill()

# ────────── 4) PROXY FILL ──────────

def proxy_ser(spec):
    return df.get(spec, qqq_series) if isinstance(spec, str) else pd.concat([df.get(t, qqq_series)*w for t, w in spec], axis=1).sum(axis=1)

filled = []
for tic, w in WEIGHTS.items():
    base = df.get(tic, pd.Series(index=df.index, dtype=float, name=tic))
    prox = proxy_ser(PROXY.get(tic, BENCH_QQQ))
    first = base.first_valid_index()
    combined = prox if first is None else base.combine_first(prox.where((base.index < first) & base.isna()))
    filled.append(combined)

filled.extend([df[BENCH_SPY], df[BENCH_QQQ]])
prices_clean = pd.concat(filled, axis=1)
prices_clean = prices_clean.loc[:, ~prices_clean.columns.duplicated()]  # dedupe
prices_clean = prices_clean.dropna().tail(window + 1)

# ────────── 5) RETURNS ──────────
rets = prices_clean.pct_change().dropna()
active_weights = {t: w for t, w in WEIGHTS.items() if t in rets.columns and w > 0}
portfolio_r = rets[list(active_weights)] @ pd.Series(active_weights)
spy_r = rets[BENCH_SPY]
qqq_r = rets[BENCH_QQQ]

# ────────── 6) METRICS ──────────
ann_ret = (1+portfolio_r).prod()**(TRADING_DAYS_YR/len(portfolio_r))-1
ann_vol = portfolio_r.std(ddof=0)*np.sqrt(TRADING_DAYS_YR)
sharpe  = np.nan if ann_vol==0 else (ann_ret-RISK_FREE_RATE)/ann_vol
cum = (1+portfolio_r).cumprod()
max_dd = (cum/cum.cummax()-1).min()
alpha = 1-CONF_LVL
var_pct = np.percentile(portfolio_r, alpha*100)
cvar_pct = portfolio_r[portfolio_r<=var_pct].mean()
beta_spy = portfolio_r.cov(spy_r)/spy_r.var()
beta_qqq = portfolio_r.cov(qqq_r)/qqq_r.var()

# ────────── 7) DASHBOARD ──────────
level = int(CONF_LVL*100)
print("\n".join([
    f"Risk Dashboard - {YEARS} years ({len(portfolio_r)} days)",
    f"Annualized Return      : {ann_ret:.2%}",
    f"Annualized Volatility  : {ann_vol:.2%}",
    f"Sharpe Ratio (rf={RISK_FREE_RATE:.0%})   : {sharpe:.2f}",
    f"Max Drawdown           : {max_dd:.2%}",
    f"1-Day VaR  ({level}%): {var_pct:.2%} (~${var_pct*PORTF_VALUE_USD:,.0f})",
    f"1-Day CVaR ({level}%): {cvar_pct:.2%} (~${cvar_pct*PORTF_VALUE_USD:,.0f})",
    f"Beta vs. S&P 500       : {beta_spy:.2f}",
    f"Beta vs. Nasdaq-100    : {beta_qqq:.2f}",
    f"❤️ DAISY ❤️"
]))

# ────────── 8) EXPORT RETURNS TO CSV ──────────
from pathlib import Path

# 1️⃣ Output folder
output_dir = Path("csv_export")
output_dir.mkdir(exist_ok=True)

# 2️⃣ Main "Returns.csv": portfolio tickers, proxy-adjusted
returns_all = pd.concat(filled, axis=1)
returns_all = returns_all.loc[:, ~returns_all.columns.duplicated()]  # Remove duplicated tickers (e.g. QQQ)
returns_all = returns_all.dropna().pct_change().dropna()
returns_all.index.name = "Date"
returns_all.to_csv(output_dir / "Returns.csv")
print(f"✅ Wrote: {output_dir/'Returns.csv'}")

# 3️⃣ Per-proxy raw constituent return files
for target, spec in PROXY.items():
    if isinstance(spec, str):
        if spec in df:
            returns_proxy = df[[spec]].pct_change().dropna()
            returns_proxy.index.name = "Date"
            returns_proxy.to_csv(output_dir / f"{target}.csv")
            print(f"✅ Wrote: {output_dir / f'{target}.csv'}")
    else:
        tickers = [t for t, _ in spec if t in df.columns]
        if tickers:
            returns_proxy = df[tickers].pct_change().dropna()
            returns_proxy.index.name = "Date"
            returns_proxy.to_csv(output_dir / f"{target}.csv")
            print(f"✅ Wrote: {output_dir / f'{target}.csv'}")

print(f"\n📁 All exports saved in: {output_dir.resolve()}")


Risk Dashboard - 6 years (1493 days)
Annualized Return      : 53.06%
Annualized Volatility  : 58.94%
Sharpe Ratio (rf=5%)   : 0.82
Max Drawdown           : -55.13%
1-Day VaR  (95%): -3.65% (~$-3,645)
1-Day CVaR (95%): -5.09% (~$-5,090)
Beta vs. S&P 500       : 1.42
Beta vs. Nasdaq-100    : 1.22
❤️ DAISY ❤️
✅ Wrote: csv_export\Returns.csv
✅ Wrote: csv_export\RDDT.csv
✅ Wrote: csv_export\HOOD.csv
✅ Wrote: csv_export\NBIS.csv
✅ Wrote: csv_export\SNOW.csv
✅ Wrote: csv_export\SHOP.csv

📁 All exports saved in: E:\STOCKS\Python\csv_export
